In [1]:
# 测试通信开销

In [4]:
import numpy as np
from utils.SPPTD import SimWorkers, CATD, S0, S1
import time
from decimal import Decimal
import random

In [25]:
def TEST(K, M, S):
    res = [0, 0, 0, 0] # worker, S_0 -> S_1, S_0 -> S_1, S_1 -> S_0
    ground_truth = [int(random.random() * 100) for m in range(M)]
    vw = SimWorkers(K, M, S, ground_truth)
    
    [S0_Data, S1_Data] = vw.ss()
    for i in S0_Data:
        res[0] += i.nbytes
    for i in S1_Data:
        res[0] += i.nbytes

    init_truth = [int(random.random() * 100) for m in range(M)]
    s0 = S0(K, M, *S0_Data)    
    [C0, C_pack1] = s0.pre_up_weight()
    res[1] +=np.array(C0).nbytes
    for i in C_pack1:
        res[1] += i.nbytes
    
    
    s1 = S1(K, M, C0, C_pack1, s0.pk, *S1_Data)
    
    C = s1.up_weight(init_truth)
    res[3] += C.nbytes
        
    C_pack2 = s0.up_truth(C)
    for i in C_pack2:
        res[2] += i.nbytes
        
    [C5, C6] = s1.up_truth(C_pack2)
    res[3] += C5.nbytes
    res[3] += C6.nbytes
    
    x = s0.resolve_xt(C5, C6)
    res[2] += np.array(x).nbytes
    return res

In [26]:
TEST(10, 20, 8)

[9600, 8080, 3440, 400]

In [27]:
t1 = []
K = 10
for M in range(20, 101, 20):
    t1.append(TEST(K, M ,8))

In [28]:
t2 = []
M = 20
for K in range(20, 101, 20):
    t2.append(TEST(K, M, 8))

In [29]:
t1

[[9600, 8080, 3440, 400],
 [19200, 16080, 6800, 720],
 [28800, 24080, 10160, 1040],
 [38400, 32080, 13520, 1360],
 [48000, 40080, 16880, 1680]]

In [30]:
t2

[[19200, 16160, 6720, 480],
 [38400, 32320, 13280, 640],
 [57600, 48480, 19840, 800],
 [76800, 64640, 26400, 960],
 [96000, 80800, 32960, 1120]]